In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from dotenv import dotenv_values
import os

memory = SqliteSaver.from_conn_string(":memory:")

In [3]:
config = dotenv_values(".env")

In [6]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [7]:
tool = TavilySearchResults(max_results=2)

In [8]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [10]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=config["OPEN_AI_KEY"])
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [11]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', id='849bcd3e-a675-40ee-b3cd-35cec480df07'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_g5Irfsbd43deADe6Uwfu30pj', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7387b42a-7c86-4c6f-8ab7-d06289075330-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_g5Irfsbd43deADe6Uwfu30pj'}])]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_g5Irfsbd43deADe6Uwfu30pj', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, resp

In [12]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='849bcd3e-a675-40ee-b3cd-35cec480df07'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'id': 'call_g5Irfsbd43deADe6Uwfu30pj', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174}}, id='run-7387b42a-7c86-4c6f-8ab7-d06289075330-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_g5Irfsbd43deADe6Uwfu30pj'}])]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '2024-07-01T06:44:13.560122+00:00'}}, parent_config=None)

In [13]:
abot.graph.get_state(thread).next

('action',)

In [18]:
# Continue after Interrupt

In [14]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_g5Irfsbd43deADe6Uwfu30pj'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://weatherspark.com/h/y/557/2024/Historical-Weather-during-2024-in-San-Francisco-California-United-States\', \'content\': \'San Francisco Temperature History 2024\\nHourly Temperature in 2024 in San Francisco\\nCompare San Francisco to another city:\\nCloud Cover in 2024 in San Francisco\\nDaily Precipitation in 2024 in San Francisco\\nObserved Weather in 2024 in San Francisco\\nHours of Daylight and Twilight in 2024 in San Francisco\\nSunrise & Sunset with Twilight and Daylight Saving Time in 2024 in San Francisco\\nSolar Elevation and Azimuth in 2024 in San Francisco\\nMoon Rise, Set & Phases in 2024 in San Francisco\\nHumidity Comfort Levels in 2024 in San Francisco\\nWind Speed in 2024 in San Francisco\\nHourly Wind Speed in 2024 in San Francisco\\nHourly Wind Direction i

In [15]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='849bcd3e-a675-40ee-b3cd-35cec480df07'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'id': 'call_g5Irfsbd43deADe6Uwfu30pj', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174}}, id='run-7387b42a-7c86-4c6f-8ab7-d06289075330-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_g5Irfsbd43deADe6Uwfu30pj'}]), ToolMessage(content='[{\'url\': \'https://weatherspark.com/h/y/557/2024/Historical-Weather-during-2024-in-San-Francisco-California-United-States\', \'content\': \'San Francisco Temperature History 2024\\nHourly Temperature in 2024

In [16]:
abot.graph.get_state(thread).next

()

In [17]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='a2b9bea2-ba88-4b40-8811-bc268ef29a2e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pqGb1RnUb9M0NCVzG5m720Wz', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-db313ac2-7ae8-4dce-986b-1938ad9cbdab-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_pqGb1RnUb9M0NCVzG5m720Wz'}])]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pqGb1RnUb9M0NCVzG5m720Wz', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': 

proceed? y


Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_pqGb1RnUb9M0NCVzG5m720Wz'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.timeanddate.com/weather/usa/los-angeles/historic?month=1&year=2024\', \'content\': \'Weather reports from January 2024 in Los Angeles, California, USA with highs and lows. Sign in. News. News Home; ... January 2024 Weather in Los Angeles — Graph °F. See Hour-by-hour Forecast for upcoming weather. ... 30.07 "Hg: 10 mi: 4:53 pm: 64 °F: Sunny.\'}, {\'url\': \'https://www.wunderground.com/hourly/us/ca/los-angeles/90053/date/2024-07-01\', \'content\': \'Los Angeles Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Los Angeles area. ... Monday 07/01 ...\'}]', name='tavily_search_results_json', id='c04dfbf7-2b3e-4d08-8a64-b1fbfceff277', tool_call_id='call_pqGb1RnUb9M0NCVzG5m720Wz')]}
{'messag

In [19]:
# Modify State

In [26]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='be2e68ff-3308-4bb7-aabc-170372e6b750'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-67536424-1b36-4d3f-ace2-0636f206d4bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}])]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': 

In [27]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='be2e68ff-3308-4bb7-aabc-170372e6b750'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}}, id='run-67536424-1b36-4d3f-ace2-0636f206d4bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}])]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '2024-07-01T07:31:59.295683+00:00'}}, parent_config=None)

In [28]:
current_values = abot.graph.get_state(thread)

In [29]:
current_values.values['messages'][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}}, id='run-67536424-1b36-4d3f-ace2-0636f206d4bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}])

In [30]:
current_values.values['messages'][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Los Angeles'},
  'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}]

In [31]:
_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Louisiana'},
  'id': _id}
]

In [32]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='be2e68ff-3308-4bb7-aabc-170372e6b750'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}}, id='run-67536424-1b36-4d3f-ace2-0636f206d4bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}])]}


{'configurable': {'thread_id': '3',
  'thread_ts': '2024-07-01T07:32:18.326828+00:00'}}

In [33]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='be2e68ff-3308-4bb7-aabc-170372e6b750'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}}, id='run-67536424-1b36-4d3f-ace2-0636f206d4bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}])]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '2024-07-01T07:32:18.326828+00:00'}}, parent_config=None)

In [34]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://world-weather.info/forecast/usa/louisiana/january-2024/\', \'content\': \'Detailed ⚡ Louisiana Weather Forecast for January 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info\'}, {\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.44, \'lon\': -91.06, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1719819159, \'localtime\': \'2024-07-01 2:32\'}, \'current\': {\'last_updated_epoch\': 1719819000, \'last_updated\': \'2024-07-01 02:30\', \'temp_c\': 15.7, \'temp_f\': 60.3, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 4.3, \'win

In [35]:
# Time Travel
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='be2e68ff-3308-4bb7-aabc-170372e6b750'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}}, id='run-67536424-1b36-4d3f-ace2-0636f206d4bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}]), ToolMessage(content='[{\'url\': \'https://world-weather.info/forecast/usa/louisiana/january-2024/\', \'content\': \'Detailed ⚡ Louisiana Weather Forecast for January 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info\'}, {\'u

In [36]:
to_replay = states[-3]

In [37]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='be2e68ff-3308-4bb7-aabc-170372e6b750'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}}, id='run-67536424-1b36-4d3f-ace2-0636f206d4bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}])]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '2024-07-01T07:32:18.326828+00:00'}}, parent_config=None)

In [38]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}
Back to the model!
{'messages': [ToolMessage(content="[{'url': 'https://weatherspark.com/h/y/11799/2024/Historical-Weather-during-2024-in-New-Orleans-Louisiana-United-States', 'content': '2024 Weather History in New Orleans Louisiana, United States. ... 30.01 inHg: Raw: KMSY 230553Z 00000KT 10SM FEW020 27/24 A3001 RMK AO2 SLP164 T02720244 10311 20272 403330244 58004 $ ... This report shows the past weather for New Orleans, providing a weather history for 2024. It features all historical weather data series we have available ...'}, {'url': 'https://www.cnn.com/2024/07/01/weather/hurricane-beryl-caribbean-landfall-monday/index.html', 'content': 'Hurricane Beryl, an extremely hazardous Category 4 storm, is set to slam into the Caribbean Monday, endangering island communities with life-threatening storm surge, violent winds and flash flooding.'}]", name=

In [39]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='be2e68ff-3308-4bb7-aabc-170372e6b750'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}}, id='run-67536424-1b36-4d3f-ace2-0636f206d4bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}])]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '2024-07-01T07:32:18.326828+00:00'}}, parent_config=None)

In [40]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']
to_replay.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in LA, accuweather'},
  'id': _id}]

In [41]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='be2e68ff-3308-4bb7-aabc-170372e6b750'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}}, id='run-67536424-1b36-4d3f-ace2-0636f206d4bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}])]}


In [42]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}
Back to the model!
{'messages': [ToolMessage(content="[{'url': 'https://www.accuweather.com/en/us/los-angeles/90012/current-weather/347625', 'content': 'Current weather in Los Angeles, CA. Check current conditions in Los Angeles, CA with radar, hourly, and more.'}, {'url': 'https://www.accuweather.com/en/us/los-angeles/90012/weather-forecast/347625', 'content': 'Los Angeles, CA Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.'}]", name='tavily_search_results_json', id='7322a29d-2d44-421e-be0f-e7c481f3923c', tool_call_id='call_w4ZUKqw5uagKk1B44YHxJU6O')]}
{'messages': [HumanMessage(content='Whats the weather in LA?', id='be2e68ff-3308-4bb7-aabc-170372e6b750'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavil

In [43]:
# Add Messages to a State at a given time
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='be2e68ff-3308-4bb7-aabc-170372e6b750'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}}, id='run-67536424-1b36-4d3f-ace2-0636f206d4bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}])]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '2024-07-01T07:32:18.326828+00:00'}}, parent_config=None)

In [44]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']

In [45]:
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="tavily_search_results_json",
    content="54 degree celcius",
)]}

In [46]:
branch_and_add = abot.graph.update_state(
    to_replay.config, 
    state_update, 
    as_node="action")

In [47]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='be2e68ff-3308-4bb7-aabc-170372e6b750'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}}, id='run-67536424-1b36-4d3f-ace2-0636f206d4bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_w4ZUKqw5uagKk1B44YHxJU6O'}]), ToolMessage(content='54 degree celcius', name='tavily_search_results_json', id='ad1f8d95-4959-4f57-af93-a5fc0936a534', tool_call_id='call_w4ZUKqw5uagKk1B44YHxJU6O'), AIMessage(content='The current weather in Louisiana is 54 degrees Celsius. Would you like me to che